<p>The following notebook provides script of scraping the data <b>ijob.am.</b>
The general sequence of steps I followed are:
    
1. [Dealing with infinite load button ("Load more") and getting all job urls](#load)
2. [Defining a function getting data of interest from job announcements](#func)  
3. [Crawling, getting and saving the data into csv](#crawl)

 <h2>1.Dealing with infinite load button ("Load more") and getting all job urls</h2> <a name="load"></a>

In [1]:
#importing necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from collections import OrderedDict

In [2]:
#opening browser
browser=webdriver.Chrome()
url_p="http://ijob.am/job-list/"
browser.get(url_p)
browser.implicitly_wait(30)

In [3]:
#dealing with "Load more" button: clicking it untill page is not refreshed
while True:
    try:
        loadMoreButton = browser.find_element_by_class_name("load_more_jobs")
        time.sleep(2)
        loadMoreButton.click()
        time.sleep(2)
    except Exception as e:
        print(e)
        break
print("Complete")
page=browser.page_source
browser.quit()

Message: element not interactable
  (Session info: chrome=77.0.3865.90)

Complete


In [4]:
#parsing the page into html and saving in bs4
page = BeautifulSoup(page,"html.parser")

In [5]:
#getting all job urls
job_desc_urls=[i.get("href") for i in page.find_all("a", class_= "job_listing-clickbox")]

 <h2>2.Defining a function getting data of interest from job announcements</h2> <a name="func"></a>

In [9]:
#getting the selected data out of job announcement
def get_all(url):
    '''function sending request to job announcement pages and returning the outlined details'''
    response=requests.get(url)
    time.sleep(2)
    page=response.text
    page=BeautifulSoup(page, "html.parser")
    company_name=page.find("li", class_="job-company").get_text()
    position=page.find("h1", class_="page-title").get_text()
    location=page.find("div", class_="job-location").get_text()
    category=page.find("div", class_="job_listing-categories").get_text()
    job_load=page.findChild("h3", class_="page-subtitle").find_next("li").get_text()
    responsibilities=page.find("div", id="jmfe-custom-job_responsibilities").get_text()
    qualifications=page.find("div", id="jmfe-custom-job_required_qualifications").get_text() 
    return company_name,position,location,category,job_load,responsibilities,qualifications

 <h2>3.Crawling, getting and saving the data into csv</h2> <a name="crawl"></a>

In [19]:
#crawling ijob.am and saving the data into final_data_ijob list
# continue is for going to next iteration regardless of the exceptions raised by the code
final_data_ijob=[]
for i in  job_desc_urls:
    try:
        all_data=get_all(i)
    except:
        continue
    final_data_ijob.append(all_data)
    
    

In [22]:
#saving in dataframe and csv file
pd.DataFrame(pd.DataFrame(final_data_ijob,columns=["company_name","position","location","category","job_load","responsibilities","qualifications"])).to_csv("Final_data_ijob.csv", index=False, header=True)